## Extract footprint information from long time-series s3l2 products.
#### 1. The nearest valid footprint and the corresponding informatio, e.g.,waveform.
#### 2. The ransac prediction ssha value corresponding to the nearest footprint (may be not the valid ssha).
#### 3. Interpolation result with two time-nearest tgauge data.
#### 4. ssha difference of two time-rearest tgauge data.


In [1]:
import sys
sys.path.append("../../utils")
import matplotlib.pyplot as plt
from get_filesPath import get_filesPath
from s3l2_extract import s3l2_extract
from tgauge_data_interp import tgauge_data_interp
import xarray as xr
import numpy as np
import os
from tiff_io import readTiff
from imgShow import imgShow


In [2]:
root_dir = '/Users/luo/OneDrive/SAR-Altimetry/sentinel3-altimetry-l2'
dir_s3al2_260 = root_dir+'/data/s3a-relative-orbit-260'
dir_s3al2_309 = root_dir+'/data/s3a-relative-orbit-309'
dir_output = root_dir + '/results'


In [3]:
## data prepare
files_path_s3a_260 = get_filesPath(base=dir_s3al2_260, key_words='enhanced_measurement_subs_preproc_mainproc_postproc.nc')
files_path_s3a_309 = get_filesPath(base=dir_s3al2_309, key_words='enhanced_measurement_subs_preproc_mainproc_postproc.nc')
files_path = files_path_s3a_260 + files_path_s3a_309

files_path = sorted(files_path, key=lambda s: s[101:])
print('s3l2 files number:', len(files_path))


s3l2 files number: 50


In [4]:
retrackers = ['ocean', 'ocog', 'ice_sheet', 'sea_ice']
for retracker in retrackers:
    ''' 1. the valid nearest footprint are vary with retrackers, thus the features (e.g., waveform) to be added to the .nc file is different with retrackers.
        2. the nearest valid footprint position may be different, but the nearest footprint position is the same 
    '''
    print('retracker:', retracker)
    for i, path_s3l2 in enumerate(files_path):
        s3_l2 = xr.open_dataset(path_s3l2)
        ## determine indexes of the nearest footprint and valid nearest footprint.
        ind_fp_near = np.argmin(s3_l2['dis_fp_tgauge'].values)
        ind_nan = np.isnan(s3_l2['ssha_land_wf_ransac_filter_'+retracker])
        s3_l2['dis_fp_tgauge'][ind_nan] = np.nan
        ind_fp_near_valid = np.nanargmin(s3_l2['dis_fp_tgauge'])
        
        ## 1. extract footprint information from s3l2, e.g., time, waveform,location...
        fp_record = s3l2_extract(s3_l2, ind_fp_near_valid)
        
        ## 2. add nearest ransac prediction variable
        ## 2.1 land randsac prediction
        fp_record['ssha_land_ransac_pred_'+retracker] = s3_l2['ssha_land_ransac_pred_'+retracker].values[ind_fp_near]
        fp_record['ssha_land_ransac_pred_'+retracker].attrs['name'] = 'land filtered ransac prediction corresponding to the nearest footprint to tgauge station'
        ## 2.2 wf randsac prediction
        fp_record['ssha_wf_ransac_pred_'+retracker] = s3_l2['ssha_wf_ransac_pred_'+retracker].values[ind_fp_near]
        fp_record['ssha_wf_ransac_pred_'+retracker].attrs['name'] = 'wf filtered ransac prediction corresponding to the nearest footprint to tgauge station'
        # 2.3 land+wf filtered ransac prediction
        fp_record['ssha_land_wf_ransac_pred_'+retracker] = s3_l2['ssha_land_wf_ransac_pred_'+retracker].values[ind_fp_near]
        fp_record['ssha_land_wf_ransac_pred_'+retracker].attrs['name'] = 'land and wf filtered ransac prediction corresponding to the nearest footprint to tgauge station'

        ## 3. add tgauge station location
        fp_record['lon_tgauge'] = s3_l2['lon_tgauge'].values
        fp_record['lat_tgauge'] = s3_l2['lat_tgauge'].values
        fp_record['tgauge_id'] = s3_l2['tgauge_id'].values
        ## 4.linear interpolation of the tgauge data
        target_time = s3_l2['time_20_ku'][ind_fp_near_valid]
        ssha_tgauge_interp, ind_time_start = tgauge_data_interp(tgauge_values=s3_l2['ssha_tgauge'], tgauge_times=s3_l2['time_tgauge'], target_time= target_time)
        ssha_tgauge_interp = ssha_tgauge_interp-1.38 ## specify to Hong Kong region
        fp_record['ssha_tgauge_interp'] = ssha_tgauge_interp.values
        fp_record['ssha_tgauge_interp'].attrs['unit'] = 'meter'
        
        ## 5. add ssha difference between two nearest times of tgauge data
        fp_record['tgauge_ssha_dif_near2'] = abs(s3_l2['ssha_tgauge'][ind_time_start]-s3_l2['ssha_tgauge'][ind_time_start+1])
        fp_record['tgauge_ssha_dif_near2'].attrs['name'] = 'ssha difference between two neareast tgauge time to the satellite data acquisition time'
        fp_record['tgauge_ssha_dif_near2'].attrs['unit'] = 'meter'
        ## concatenation of the obtained data
        if i == 0:
            fp_records = fp_record
            continue
        fp_records = xr.concat([fp_records, fp_record], dim='time_20_ku', coords='all')
        print('time:', fp_record['time_20_ku'].values, end=' ')
        print('ssha_tgauge: %.3f' % fp_record['ssha_tgauge_interp'].values, end=' ')
        print('ssha_estimate: %.3f' % fp_record['ssha_estimate_'+retracker].values)
    ## writh to new .nc file
    path_result = os.path.join(dir_output, 'result_final_'+retracker+'.nc')
    fp_records.to_netcdf(path=path_result)


retracker: ocean
time: 2019-01-24T14:08:49.746136704 ssha_tgauge: 0.966 ssha_estimate: 0.854
time: 2019-02-17T02:37:22.816082432 ssha_tgauge: 0.044 ssha_estimate: -0.003
time: 2019-02-20T14:08:50.092905984 ssha_tgauge: 1.057 ssha_estimate: 0.975
time: 2019-03-16T02:37:19.856276096 ssha_tgauge: 0.270 ssha_estimate: -0.036
time: 2019-03-19T14:08:48.966854272 ssha_tgauge: 0.634 ssha_estimate: 0.629
time: 2019-04-12T02:37:25.660643840 ssha_tgauge: 0.635 ssha_estimate: 0.560
time: 2019-04-15T14:08:53.921298560 ssha_tgauge: -0.122 ssha_estimate: -0.314
time: 2019-05-09T02:37:28.304749696 ssha_tgauge: 1.031 ssha_estimate: 1.046
time: 2019-05-12T14:08:56.330205440 ssha_tgauge: -0.759 ssha_estimate: -0.754
time: 2019-06-05T02:37:27.693831168 ssha_tgauge: 0.903 ssha_estimate: 0.673
time: 2019-06-08T14:08:55.368344832 ssha_tgauge: -0.760 ssha_estimate: -0.886
time: 2019-07-02T02:37:27.366581248 ssha_tgauge: 0.730 ssha_estimate: 0.633
time: 2019-07-05T14:08:55.213489536 ssha_tgauge: -0.409 ssha_es

In [6]:
# s3_l2
# fp_records


<xarray.Dataset>
Dimensions:                                (echo_sample_ind: 128, lat_tgauge: 5, lon_tgauge: 5, tgauge_id: 1, time_20_ku: 50)
Coordinates:
  * time_20_ku                             (time_20_ku) datetime64[ns] 2019-0...
    lat_20_ku                              (time_20_ku) float64 22.19 ... 22.22
    lon_20_ku                              (time_20_ku) float64 114.2 ... 114.3
  * lon_tgauge                             (lon_tgauge) float64 114.2 ... 114.0
  * lat_tgauge                             (lat_tgauge) float64 22.3 ... 22.49
  * tgauge_id                              (tgauge_id) int64 0
Dimensions without coordinates: echo_sample_ind
Data variables: (12/137)
    UTC_day_20_ku                          (time_20_ku) datetime64[ns] 2019-0...
    UTC_sec_20_ku                          (time_20_ku) float64 9.442e+03 ......
    UTC_time_1hz_20_ku                     (time_20_ku) datetime64[ns] 2019-0...
    lat_cor_20_ku                          (time_20_ku) float64 22.19 ... 22.22
    lon_cor_20_ku                          (time_20_ku) float64 114.2 ... 114.3
    surf_type_20_ku                        (time_20_ku) float64 0.0 0.0 ... 0.0
    ...                                     ...
    ssha_land_wf_ransac_filter_sea_ice     (time_20_ku) float64 0.2864 ... 0....
    ssha_land_ransac_pred_sea_ice          (time_20_ku) float64 0.4473 ... 0....
    ssha_wf_ransac_pred_sea_ice            (time_20_ku) float64 0.4968 ... 0....
    ssha_land_wf_ransac_pred_sea_ice       (time_20_ku) float64 0.4968 ... 0....
    ssha_tgauge_interp                     (time_20_ku) float64 0.2951 ... 0....
    tgauge_ssha_dif_near2                  (time_20_ku) float64 0.04 ... 0.28
Attributes: (12/44)
    Conventions:                       CF-1.6
    title:                             IPF SRAL/MWR Level 2 Measurement
    mission_name:                      Sentinel 3A
    altimeter_sensor_name:             SRAL
    radiometer_sensor_name:            MWR
    gnss_sensor_name:                  GNSS
    ...                                ...
    xref_iono_data:                    S3A_SR_2_RGI_AX_20190121T000000_201901...
    xref_mog2d_data:                   S3__SR_2_RMO_AX_20190121T000000_201901...
    xref_seaice_concentration:         S3__SR_2_SIC_AX_20190120T000000_201901...
    xref_altimeter_ltm:                S3A_SR_1_CA1LAX_20000101T000000_201909...
    xref_doris_uso:                    S3A_SR_1_USO_AX_20160223T195017_201912...
    xref_time_correlation:             S3A_AX___FRO_AX_20190121T000000_201901...